<a href="https://colab.research.google.com/github/qzhou306/Bee_CV/blob/master/Test_YoloFastest_2_in_1_Qifang_complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Connect google drive**

In [ ]:
!pip install ipython-autotime
%load_ext autotime

# Check if NVIDIA GPU is enabled
!nvidia-smi

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1831 sha256=9efae9e9ef99328b5563709629902a5204c6e6951e1b1c259d660bfe182adf69
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime
Thu Sep 24 21:28:11 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'02AUG KL ORDAMS 2人 票号 副本 (1).docx'
'02AUG KL ORDAMS 2人 票号 副本.docx'
 1_ALSWR.png
'1 class files'
 backup
 bee
'Colab Notebooks'
'Cover Letter ADP.docx'
 cudnn-10.1-linux-x64-v8.0.2.39.tgz
 darknet
'Gas Mileage.ipynb'
'How to get started with Drive.pdf'
 HW1_ISYE8803.ipynb
 i-539a-XUE.pdf
 i-539-DENG.pdf
 Image_comparison.ipynb
 IMG_8227.jpg
'interview tips.gdoc'
'ISYE 6740 Final.ipynb'
'ISYE 8803 HW3 Q1.ipynb'
'Lawrence coding cert 1.jpg'
'MIdterm 1 Q1 Qifang Zhou.ipynb'
'My Drive'
'New folder'
'Qifang Zhou 2020_G.doc'
'Qifang Zhou_Sep Data.docx'
'Qifang Zhou_Sep Data.pdf'
'Qifang Zhou_Sep.pdf'
 Qnote
 results.avi
 results.mp4
'RPubs - Adaptive LASSO Examples.png'
 Sensata.gdoc
'Sprayer Qnotes SAP w999.xlsx'
 TensorFlow-2.x-YOLOv3-master
'Test Codes.ipynb'
 Traveller+public+health+declaration.pdf
 Untitled0.ipynb
'week3 Car detection for Autonomous Driving .png'
'Weld Qnote Classification.ipynb'
 yolov3
'zgle-fork TensorFlow-2.x-YOLOv3: YOLOv3.png'
 Zhou_Qifa

**1) Clone, configure & compile Darknet**



In [ ]:
# Clone
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14309, done.
remote: Total 14309 (delta 0), reused 0 (delta 0), pack-reused 14309
Receiving objects: 100% (14309/14309), 12.86 MiB | 24.34 MiB/s, done.
Resolving deltas: 100% (9766/9766), done.
time: 2.42 s


In [ ]:
# Configure
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
# comment this line for other YOLOV3
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

/content/darknet
time: 441 ms


In [ ]:
# Compile
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
  

**2) Configure yolov3.cfg file**

In [ ]:
# Make a copy of yolo-fastest.cfg
!cp /mydrive/yolov3/yolo-fastest.cfg cfg/yolo-fastest_training.cfg 

time: 411 ms


In [ ]:
# Change lines in yolov3.cfg file
'''
!sed -i 's/batch=1/batch=64/' cfg/yolo-fastest_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolo-fastest_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolo-fastest_training.cfg
!sed -i '610 s@classes=80@classes=2@' cfg/yolo-fastest_training.cfg
!sed -i '696 s@classes=80@classes=2@' cfg/yolo-fastest_training.cfg
!sed -i '783 s@classes=80@classes=2@' cfg/yolo-fastest_training.cfg
!sed -i '603 s@filters=255@filters=21@' cfg/yolo-fastest_training.cfg
!sed -i '689 s@filters=255@filters=21@' cfg/yolo-fastest_training.cfg
!sed -i '776 s@filters=255@filters=21@' cfg/yolo-fastest_training.cfg
'''

"\n!sed -i 's/batch=1/batch=64/' cfg/yolo-fastest_training.cfg\n!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolo-fastest_training.cfg\n!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolo-fastest_training.cfg\n!sed -i '610 s@classes=80@classes=2@' cfg/yolo-fastest_training.cfg\n!sed -i '696 s@classes=80@classes=2@' cfg/yolo-fastest_training.cfg\n!sed -i '783 s@classes=80@classes=2@' cfg/yolo-fastest_training.cfg\n!sed -i '603 s@filters=255@filters=21@' cfg/yolo-fastest_training.cfg\n!sed -i '689 s@filters=255@filters=21@' cfg/yolo-fastest_training.cfg\n!sed -i '776 s@filters=255@filters=21@' cfg/yolo-fastest_training.cfg\n"

time: 3.87 ms


**3) Create .names and .data files**

In [ ]:
# step 1 - Bee
!echo -e 'Bee' > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data

# step 2 - Pollen
!echo -e 'Pollen' > data/objPollen.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/objPollen.names\nbackup = /mydrive/yolov3' > data/objPollen.data


time: 433 ms


**4) Save yolov3_training.cfg and obj.names files in Google drive**

In [ ]:
#!cp cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_testing.cfg
!cp data/obj.names /mydrive/yolov3/classes.txt

time: 310 ms


**5) Create a folder and unzip image dataset**

In [ ]:
!rm -rf data/obj
!mkdir -p data/obj
!unzip /mydrive/yolov3/images_bee.zip -d data/obj

Archive:  /mydrive/yolov3/images_bee.zip
   creating: data/obj/b_1classes_derived/
  inflating: data/obj/b_1classes_derived/0003.jpg  
  inflating: data/obj/b_1classes_derived/0003.txt  
  inflating: data/obj/b_1classes_derived/0013.jpg  
  inflating: data/obj/b_1classes_derived/0013.txt  
  inflating: data/obj/b_1classes_derived/0023.jpg  
  inflating: data/obj/b_1classes_derived/0023.txt  
  inflating: data/obj/b_1classes_derived/0033.jpg  
  inflating: data/obj/b_1classes_derived/0033.txt  
  inflating: data/obj/b_1classes_derived/0043.jpg  
  inflating: data/obj/b_1classes_derived/0043.txt  
  inflating: data/obj/b_1classes_derived/0053.jpg  
  inflating: data/obj/b_1classes_derived/0053.txt  
  inflating: data/obj/b_1classes_derived/0063.jpg  
  inflating: data/obj/b_1classes_derived/0063.txt  
  inflating: data/obj/b_1classes_derived/0073.jpg  
  inflating: data/obj/b_1classes_derived/0073.txt  
  inflating: data/obj/b_1classes_derived/0083.jpg  
  inflating: data/obj/b_1classes_

**6) Create train.txt file**

In [ ]:
import glob
import random

img_list = glob.glob("data/obj/b_1classes_derived/*.jpg")
random.shuffle(img_list)
with open("data/train.txt", "w") as f:
    f.write("\n".join(img_list[:int(len(img_list)*0.7)]))

time: 7.52 ms


In [ ]:
with open("data/test.txt", "w") as f:
    f.write("\n".join(img_list[int(len(img_list)*0.7):]))

time: 2.48 ms


**7) Download pre-trained weights for the convolutional layers file**

In [ ]:
#!wget https://pjreddie.com/media/files/darknet53.conv.74


time: 1.15 ms


Generate a pre-trained model for the initialization of the model backbone


In [ ]:
# # Make a copy of yolo-fastest.weights
#!cp /mydrive/yolov3/yolo-fastest.weights backup/yolo-fastest.weights

time: 441 µs


In [ ]:
#!./darknet partial cfg/yolo-fastest_training.cfg backup/yolo-fastest.weights yolo-fastest.conv.109 109

time: 1.37 ms


**8) Start training**

In [ ]:
#!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show
# Uncomment below and comment above to re-start your training from last saved weights
#!./darknet detector train data/obj.data cfg/yolo-fastest_training.cfg yolo-fastest.conv.109  -dont_show

time: 853 µs


Object Detection

In [ ]:
# derive the paths to the YOLO weights and model configuration

objPath="data/obj.data" # for step 1 bee
objPollenPath="data/objPollen.data" # for step 2 pollen
beeweightsPath = "/mydrive/yolov3/backup/one_class_bee_yolo_fastest_weights/yolo-fastest_training_final.weights"  # for step 1 bee
pollenweightsPath = "/mydrive/yolov3/backup/one_class_pollen_yolo_fastest_weights/yolo-fastest_training_final.weights"  # for step 2 pollen
configPath = "/mydrive/yolov3/yolo-fastest.cfg"  # common config file



time: 2.22 ms


Evaluation

In [ ]:
# define helper functions
import cv2
import matplotlib.pyplot as plt
import os
import glob
import numpy as np
import json
import time
%matplotlib inline

def imShow(path):



  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

Convert video to image: save all images vs. save it in a 3D array and get overwritten every frame? 

In [ ]:
# capture images from video and save all images on google drive - slow
'''
videoPath = "/mydrive/yolov3/Cheng_1_trim.mp4"
cap = cv2.VideoCapture(videoPath)
img_id = 1

while True:
    _,frame = cap.read()
    filename = 'testimg'+str(img_id)+'.jpg'
    img_id +=1
    cv2.imwrite(filename, frame)
cap.release()
cv2.destroyAllWindows()
'''

'\nvideoPath = "/mydrive/yolov3/Cheng_1_trim.mp4"\ncap = cv2.VideoCapture(videoPath)\nimg_id = 1\n\nwhile True:\n    _,frame = cap.read()\n    filename = \'testimg\'+str(img_id)+\'.jpg\'\n    img_id +=1\n    cv2.imwrite(filename, frame)\ncap.release()\ncv2.destroyAllWindows()\n'

time: 4.11 ms


In [ ]:
# This stops 'Run all' at this cell by causing an error
#assert False

time: 583 µs


Run YOLO big image model to generate json files for bounding boxes and probability 

In [ ]:
def json_extract(obj, key):
    """Recursively fetch values from nested JSON."""
    arr = []

    def extract(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    values = extract(obj, arr, key)
    return values

time: 20.9 ms


In [ ]:
# crop images(3D array) to bee chips
def crop_image(imgName, imgData, bee_arr):
  bee_id = 1
  imgWidth = imgData.shape[1]
  imgHeight = imgData.shape[0]
  for i in bee_arr:
    filename = '/mydrive/yolov3/2_in_1_img/bee_chips/' + imgName +'_bee'+str(bee_id)+'.jpg'
    bee_id +=1
    w = int(i[2]*imgWidth*1.2)
    h = int(i[3]*imgHeight*1.2)
    x = int(i[0]*imgWidth)
    y = int(i[1]*imgHeight) 
    #print('x', x, 'y', y, 'w', w, 'h', h)
    #bbox_color_bgr_sacs=(255,0,255)
    #line_thickness=2
    bee_img = imgData[max(0, int(y-h/2)) : min(imgHeight, int(y+h/2)), max(0, int(x-w/2)) : min(imgWidth, int(x+w/2)), :]
    #cv2.rectangle(imgData, (x,y), (x+w, y+h), bbox_color_bgr_sacs, line_thickness)
    plt.imsave(filename, bee_img)

time: 46.2 ms


In [ ]:
# outpout json file for single image
# !./darknet detector test <path to .data file> <path to config> <path to weights> <path to image>
img_list = glob.glob("/mydrive/yolov3/2_in_1_img/*.jpg")

for img in img_list:
  ts = time.time()
  print(ts)
  #img = "/mydrive/yolov3/2_in_1_img/testimg1.jpg"
  imgName = img[27:-4]
  imgData = plt.imread(img)
  !./darknet detector test $objPath $configPath $beeweightsPath $img -ext_output -dont_show -out result_bee.json
  ts = time.time()
  print(ts)
  # Open the json file
  with open('/content/darknet/result_bee.json') as f:
    data = json.load(f)
  data = data[0]
  # Find every instance of `name` in a Python dictionary.
  center_x = json_extract(data, 'center_x')
  center_y = json_extract(data, 'center_y')
  width = json_extract(data, 'width')
  height = json_extract(data, 'height')
  confidence = json_extract(data, 'confidence')
  bee_arr = np.column_stack([center_x, center_y, width, height, confidence])
  # crop image and generate individual bee chip images for step 2
  crop_image(imgName, imgData, bee_arr)
  ts = time.time()
  print(ts)
  # create path for chip images to load into step 2, probably can be more concise 
  img_list=glob.glob("/mydrive/yolov3/2_in_1_img/bee_chips/*.jpg")
  with open("data/pollen.txt", "w") as f:
    f.write("\n".join(img_list))
  val_img_list="data/pollen.txt"
  !cat $val_img_list | wc -l
  #Multiple Images at Once - Step 2 Pollen detection from all individual chip bee images
  !./darknet detector test $objPollenPath $configPath $pollenweightsPath -ext_output -dont_show -out result_pollen.json <$val_img_list> result_pollen.txt


1600983009.0465918
 CUDA-version: 10010 (10010), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv      8       3 x 3/ 2    320 x 320 x   3 ->  160 x 160 x   8 0.011 BF
   1 conv      8       1 x 1/ 1    160 x 160 x   8 ->  160 x 160 x   8 0.003 BF
   2 conv      8/   8  3 x 3/ 1    160 x 160 x   8 ->  160 x 160 x   8 0.004 BF
   3 conv      4       1 x 1/ 1    160 x 160 x   8 ->  160 x 160 x   4 0.002 BF
   4 conv      8       1 x 1/ 1    160 x 160 x   4 ->  160 x 160 x   8 0.002 BF
   5 conv      8/   8  3 x 3/ 1    160 x 160 x   8 ->  160 x 160 x   8 0.004 BF
   6 conv      4       1 x 1/ 1    160 x 160 x   8 ->  160 x 160 x   4 0.002 BF
   7 dropout    p = 0.150        102400  ->   102400
   8 Shortcut Layer: 3,  wt = 0, wn = 0, outpu

In [ ]:
assert False

In [ ]:
with open('/content/darknet/result_bee.json') as f:
  data = json.load(f)
data = data[0]



In [ ]:
#from extract import json_extract

# Find every instance of `name` in a Python dictionary.
center_x = json_extract(data, 'center_x')
center_y = json_extract(data, 'center_y')
width = json_extract(data, 'width')
height = json_extract(data, 'height')
confidence = json_extract(data, 'confidence')
bee_arr = np.column_stack([center_x, center_y, width, height, confidence])

print(bee_arr)


In [ ]:
# The most recent detections are always saved to 'predictions.jpg'
imShow('predictions.jpg')

AttributeError: ignored

In [ ]:
# detect in a video
#videoPath = "/mydrive/yolov3/Cheng_1.mp4"
#outPath="results.avi"
#!./darknet detector demo $objPath $configPath $beeweightsPath -dont_show $videoPath #-i 0 -out_filename $outPath
# !./darknet detector test $objPath $configPath $weightsPath data/obj/images/img00470.jpg

In [ ]:
# download the video with detections shown
#download(outPath)

In [ ]:
# create objPollen.names and objPollen.data for step 2 detection
!echo -e 'Pollen' > data/objPollen.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/objPollen.names\nbackup = /mydrive/yolov3' > data/objPollen.data
objPollenPath="data/objPollen.data"

In [ ]:
#val_img_list="data/test.txt"
#!cat $val_img_list | wc -l


img_list=glob.glob("/mydrive/yolov3/2_in_1_img/bee_chips/*.jpg")
#print(img_list)

with open("data/pollen.txt", "w") as f:
    f.write("\n".join(img_list))
val_img_list="data/pollen.txt"
!cat $val_img_list | wc -l


In [ ]:
#Multiple Images at Once

!./darknet detector test $objPollenPath $configPath $pollenweightsPath -ext_output -dont_show -out result_pollen.json <$val_img_list> result_pollen.txt

In [ ]:
assert False

In [ ]:
!./darknet detector test $objPollenPath $configPath $pollenweightsPath /mydrive/yolov3/2_in_1_img/bee_chips/bee1.jpg -ext_output -dont_show -out result_pollen.json

In [ ]:
download('result.json')

In [ ]:
download('result.txt')

In [ ]:
imShow('predictions.jpg')

In [ ]:
!cp results.avi /mydrive

In [ ]:
#show mp4 online (don't work yet. it may due to the large size of the mp4)
!pip install -U kora
from kora.drive import upload_public
url = upload_public('/mydrive/results.avi')
from IPython.display import HTML
HTML(f"""<video src={url} width=500 controls/>""")

In [ ]:
# convert avi to mp4
!ffmpeg -i '/mydrive/results.avi' '/mydrive/results.mp4'

In [ ]:
#show mp4 online (don't work yet. it may due to the large size of the mp4)
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/mydrive/results.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

In [ ]:
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

Get mAP result from test data

In [ ]:
!./darknet detector map data/obj.data cfg/yolo-fastest_training.cfg /mydrive/yolov3/yolo-fastest_training_last.weights